# Visualise the activation of convnet filters

In this notebook we will plot all the filters of all the layers in a convolutional network, and try to get an impression of what they do. 

The notebook is heavily inspired by [this one](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter09_part03_interpreting-what-convnets-learn.ipynb), written by the author of Keras. If you want to visualise a slightly more complicated neural network, please run through this one as well. Here, in our simpler case, we look at a MNIST digit classifier, like the one from notebook 1.

### Setup

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### Load data

We load the MNIST data, as before:

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Define the model

This time, let us use the functional model API to define the network. The structure is identical to notebook 1, but all layers are Keras functions, being applied to the output from the layer before.

In [ ]:
inputs = keras.Input(shape=input_shape)
x = keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(inputs)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.3)(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

# Define the Model instance
convnet = keras.Model(inputs=inputs, outputs=outputs)

# Print it
convnet.summary()

### Train the model

### <span style="color: red;">Exercise:<span>

Compile and train the model.

In [ ]:
batch_size = 128 
epochs = 10 

#convnet.compile(...)

#convnet.fit(...)


### Select an example image

Pick the fourth image in `X_test`, and plot it.

In [ ]:
img = X_test[4]

plt.axis("off")
plt.imshow(img)
plt.show()

**Note**: These are grayscale (black-and-white) images, but the default colormap adds some color to them, to make different values easier to perceive. In case you want to plot them in their true grayscale glory, use

```
plt.imshow(img, cmap='gray')
```

in the code above (and below).

### Predict the number

To run the model on our example image, we have to add the batch axis, so that its shape is (1, 28, 28, 1).

The `predict` function outputs the predictions for each class 0-9.


In [ ]:
img_tensor = tf.expand_dims(img, axis=0)

preds = convnet.predict(img_tensor)[0]

for i in range(len(preds)):
    print(f'{i}: score = {preds[i]:.4f}')

print()
print(f'Predicted number: {tf.argmax(preds)}')

### Get the layer activations

Now, we'll create a new `Model` instance that returns the activations of the convolution (`Conv2D`) and max-pooling (`MaxPooling2D`) layers.

In [ ]:
layer_outputs = []
layer_names = []
for layer in convnet.layers:
    print(layer)
    if isinstance(layer, (keras.layers.Conv2D, keras.layers.MaxPooling2D)):
        layer_outputs.append(layer.output)
        layer_names.append(layer.name)
activation_model = keras.Model(inputs=convnet.input, outputs=layer_outputs)

Run it to get the activations!

In [ ]:
activations = activation_model.predict(img_tensor)

Plot the activations of the fifth filter of the first layer:

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(first_layer_activation[0, :, :, 5])

### Plot all activations

After running the cell below, right-click the output and select "Disable Scrolling for Outputs", in order to show all plots below each other.

In [ ]:
images_per_row = 8
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]
    n_cols = n_features // images_per_row
    display_grid = np.zeros(((size + 1) * n_cols - 1,
                             images_per_row * (size + 1) - 1))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_index = col * images_per_row + row
            channel_image = layer_activation[0, :, :, channel_index].copy()
            display_grid[
                col * (size + 1): (col + 1) * size + col,
                row * (size + 1) : (row + 1) * size + row] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.axis("off")
    plt.imshow(display_grid, aspect="auto", cmap="viridis")

### <span style="color: red;">Exercise:<span>

Try to make sense of the plots, by changing the input image to different ones:

- What do the `conv2d_*` layers do?
- What do the `max_pooling_*` layers do?
- Does the information in the successive layers become more clear or less clear?
- Can you relate the pixels in the final layer to the different number predictions?
